In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('turkish_data.csv')

In [3]:
df.head(2)

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...


In [4]:
print(f"Total number of records present in dataset: {df.shape[0]}")

Total number of records present in dataset: 4900


This Dataset consist of news with different category like 'dunya'(word), siyaset(Politics), ekonomi(economics) etc. 

In [5]:
print("Total different category in Datatset:")
print(df['category'].unique())

Total different category in Datatset:
['siyaset ' 'dunya ' 'ekonomi ' 'kultur ' 'saglik ' 'spor ' 'teknoloji ']


In [6]:
#uncomment belwo line to install ktrain 
#!pip install ktrain

In [7]:
#import required packaged
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import re

import nltk
nltk.download('words')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()


import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\BRAJ\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BRAJ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def dataset_cleaning(df_data):
    """This function helps to remove row with missing value or if there is any dupicate records"""
    df_data = df_data.dropna()
    df_data = df_data.drop_duplicates()
    df_data = df_data.reset_index(drop=True)
    return df_data

def text_cleaning(text):
    """This function helps to clean a text after removing stop words, short words, special character,
    any link present and use stemmer to provide near to root word"""
    stop = set(stopwords.words("turkish"))
    text = text.lower()
    text = re.sub('[^abcçdefgğhıijklmnoöprsştuüvyzmi̇z]',' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = " ".join([stemmer.stem(word) for word in text.split() if (word not in stop) and len(word)>1])
    return text

In [9]:
dataset = dataset_cleaning(df)

In [10]:
dataset.shape

(4540, 2)

In [11]:
dataset['cleaned_text'] = dataset['text'].apply(lambda x:text_cleaning(x))
dataset.head(2)

,category,text,cleaned_text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...,milyon ön seç vaadi mhp nin olağan büyük kurul...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...,mesut yılmaz yüç divan ceza alabilir prof dr s...


# Split Dataset into Training and Test

In [12]:
#split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(np.array(dataset['cleaned_text']), np.array(dataset['category']),
                                                   test_size = 0.20, random_state=42, stratify = dataset['category'])

# Model Tuning

Electra model ref: https://huggingface.co/dbmdz/electra-base-turkish-cased-discriminator
i am going to use ELECTRA instead of BERT as it peformed beter with high GLEU score in different research. Electra perfomance is better when you have computing resource limitation. It takes less training time comparing to BERT. 

In [13]:

MODEL_NAME = 'dbmdz/electra-base-turkish-cased-discriminator'
t= text.Transformer(MODEL_NAME, maxlen=500, classes= dataset['category'].unique())
train = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
leaner = ktrain.get_learner(model, train_data = train, val_data = val, batch_size=20)

preprocessing train...
language: tr
train sequence lengths:
	mean : 233
	95percentile : 559
	99percentile : 896


Is Multi-Label? False
preprocessing test...
language: tr
test sequence lengths:
	mean : 240
	95percentile : 601
	99percentile : 846


In [14]:
#leaner.lr_find() #to find good leaning rate

In [ ]:
leaner.fit_onecycle(5e-5, 2) 